<a href="https://colab.research.google.com/github/LaxmanGhadi/ColabProjectsBeginners/blob/main/DL%20Project%206.%20B%26WToColor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import tensorflow as tf
import os
import cv2
from keras.models import Sequential, Model
from tensorflow.keras.utils import img_to_array, load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.layers import Conv2D,UpSampling2D, Input
from skimage.color import rgb2lab, lab2rgb , gray2rgb
from skimage.transform import resize
from skimage.io import imsave

Transfer Learning : Taking a pre-existing model and making changes in layers to make add new functionalities

in this eg. we are creating a new model and assigning it the initial layers of VGG16 model and add new custom layers in back which will help serve it's new purpose


This is a encoder-decoder application so we are taking encoder from VGG16 and adding new decoder

In [ ]:
vgmodel = VGG16()

In [ ]:
vgmodel.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

Sequential helps to make/stack layers in a model . Sequential model should only be used when layers have single input and single output

In [ ]:
newmodel = keras.Sequential()#this is an empty model , the layers are added in  ""()""

now we remove layers form vgg16 and add those to newmodel and then add new layers that will hlp
learn it's new task

In [ ]:
for i, layer in enumerate(vgmodel.layers): #enumerate -  iterate through the list and return a new array that has each list grouped with it's index eg [(0, layer1)..]
  if i<19 :
    newmodel.add(layer)

In [ ]:
newmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
for layer in newmodel.layers :
  layer.trainable = False

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d aayush9753/image-colorization-dataset

image-colorization-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref  =zipfile.ZipFile('/content/image-colorization-dataset.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
count  = 0
dir_path = '/content/data/test_color'
for path in os.scandir(dir_path):
  if path.is_file():
    count +=1
print(count)

#there are 5000 imgs in train color & BW each & 739 test imgs in Color and BW each

739


In [ ]:
train_datagen = ImageDataGenerator (rescale = 1. /225)

In [ ]:
train_Color_path = '/content/train'
train_BW_path = '/content/data/train_black'

In [ ]:
train =  train_datagen.flow_from_directory(train_Color_path,
                                           target_size = (224,224),#Vggmodel requires input of 224
                                           batch_size = 32,
                                          class_mode= None)

Found 5000 images belonging to 1 classes.


In [ ]:
train[0].shape

(32, 224, 224, 3)

In [ ]:
X = []
Y = []

for img in train[0]:
  try :
    lab = rgb2lab(img)
    X.append(lab[:,:,0])# fill X with L channel (light) info
    Y.append(lab[:,:,1:]/128)# fill Y with A and B canel info
  except:
    print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,))
print(X.shape)
print(Y.shape)

(32, 224, 224, 1)
(32, 224, 224, 2)


In [ ]:
vggfeatures = []
for i, sample in enumerate(X):
  sample =  gray2rgb(sample)
  sample = sample.reshape((1,224,224,3))
  prediction  =  newmodel.predict(sample)
  prediction = prediction.reshape((7,7,512))
  vggfeatures.append(prediction)
vggfeatures = np.array(vggfeatures)
print(vggfeatures.shape)


1/1 [==============================] - 0s 31ms/step
(32, 7, 7, 512)


In [ ]:
print("Y_shape:",Y.shape,"Y_size:",Y.size)

Y_shape: (32, 224, 224, 2) Y_size: 3211264


In [ ]:
model = Sequential()

model.add(Conv2D(256,(3,3), activation = 'relu', padding  = 'same', input_shape = (7,7,512)))
model.add(Conv2D(128,(3,3), activation = 'relu', padding = 'same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(64,(3,3), activation = 'relu', padding = 'same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32,(3,3), activation = 'relu', padding = 'same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(16,(3,3), activation = 'relu', padding = 'same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(2,(3,3), activation = 'tanh', padding = 'same'))
model.add(UpSampling2D((2,2)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 7, 7, 256)         1179904   
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         295040    
                                                                 
 up_sampling2d (UpSampling2D  (None, 14, 14, 128)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        73792     
                                                                 
 up_sampling2d_1 (UpSampling  (None, 28, 28, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 32)       

In [ ]:

model.compile(optimizer = 'adam',loss='mse' , metrics = ['accuracy']  )


In [ ]:
model.fit(vggfeatures , Y , verbose = 1 ,epochs=1000, batch_size=32)
model.save('colorize_autoencoder_VGG16.model')

Epoch 1/1000
1/1 [==============================] - 4s 4s/step - loss: 0.7450 - accuracy: 0.5342
Epoch 2/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.6693 - accuracy: 0.6337
Epoch 3/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.8951 - accuracy: 0.6352
Epoch 4/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.7280 - accuracy: 0.6327
Epoch 5/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.8966 - accuracy: 0.4763
Epoch 6/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.8942 - accuracy: 0.3991
Epoch 7/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.8279 - accuracy: 0.3788
Epoch 8/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.9563 - accuracy: 0.3642
Epoch 9/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.9832 - accuracy: 0.3634
Epoch 10/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.9726 - accuracy: 0.3636
Epoch 11/10

In [ ]:
def plotImage(imageArr):
  for img in zip(imageArr):
    plt.figure(figsize = (5,5))
    plt.imshow(img)
    plt.show()

In [ ]:
import os
from PIL import Image
model2 = tf.keras.models.load_model('/content/colorize_autoencoder_VGG16.model',
                                    custom_objects= None,
                                    compile = True)

In [ ]:
test_path = '/content/data/test_color/'
files = os.listdir(test_path)
i=0
for index , file in enumerate(files):
    test = img_to_array(load_img(test_path+file))
    test = resize(test,(224,224), anti_aliasing = True)
    test *= 1.0/255
    lab = rgb2lab(test)
    l = lab[:,:,0]
    L = gray2rgb(l)
    L = L.reshape((1,224,224,3))
    vggpred = newmodel.predict(L)
    ab = model.predict(vggpred)
    ab = ab*128
    cur = np.zeros((224,224,3))
    cur[:,:,0] = l
    cur[:,:,1:] = ab
    imsave('/content/AAA/'+str(index)+".jpg",lab2rgb(cur))

1/1 [==============================] - 0s 304ms/step


1/1 [==============================] - 0s 18ms/step


/usr/local/lib/python3.10/dist-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 499 pixels
  return func(*args, **kwargs)


1/1 [==============================] - 0s 19ms/step


/usr/local/lib/python3.10/dist-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 173 pixels
  return func(*args, **kwargs)


...

1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 24ms/step


/usr/local/lib/python3.10/dist-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 576 pixels
  return func(*args, **kwargs)


1/1 [==============================] - 0s 24ms/step


KeyboardInterrupt: ignored

In [ ]:
del(model)